In [3]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Plots, StatsPlots, Distributions

## ____________________________________________________________________

## Import Datasets (Outputted Probabilities and Returns on $100 Bets)

In [69]:
# # add actual data here
# xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\xgb_cv_pred.csv",DataFrame)
# rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame)
# lgbm_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\lgbm_cv_pred.csv",DataFrame);
#logreg_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\logreg_pred2.csv",DataFrame);
new_xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\new_xgb_pred2.csv",DataFrame);
# rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame);
returns = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\group_stage_betting_odds_final_cleaned.csv",DataFrame);

# # add actual data here
#new_xgb_preds = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/notebooks/new_xgb_pred.csv",DataFrame);
#rf_preds = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/notebooks/rf_pred.csv",DataFrame);
#returns = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/data/group_stage_betting_odds_final_cleaned.csv",DataFrame);

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,Mexico,0.549451,2.95,6.12,0.64539,0.253165,0.140449
2,Argentina,Saudi Arabia,0.120048,8.21,24.52,0.892819,0.108578,0.039185
3,Australia,Denmark,6.38,3.88,0.429185,0.135501,0.204918,0.6997
4,Belgium,Canada,0.900901,2.45,3.51,0.526066,0.289855,0.221729
5,Belgium,Morocco,0.581395,3.21,4.92,0.632353,0.23753,0.168919
6,Brazil,Serbia,0.5,3.28,6.6,0.666667,0.233645,0.131579
7,Brazil,Switzerland,0.469484,3.57,6.23,0.680511,0.218818,0.138313
8,Cameroon,Brazil,7.93,4.32,0.359712,0.111982,0.18797,0.73545
9,Cameroon,Serbia,4.3,2.77,0.70922,0.188679,0.265252,0.585062


In [70]:
returns_matrix = Matrix(returns[:,3:5]);
# # matrix of probabilities for model's predictions
# xgb_probs = Matrix(xgb_preds[:,3:5]);
#rf_probs = Matrix(rf_preds[:,3:5]);
# lgbm_probs = Matrix(lgbm_preds[:,3:5]);
#logreg_probs = Matrix(logreg_preds[:,3:5]);
new_xgb_probs = Matrix(new_xgb_preds[:,3:5]);

In [71]:
new_xgb_probs[1:5,:]

5×3 Matrix{Float64}:
 0.998929  0.0006162   0.000455174
 0.979123  0.0143582   0.00651851
 0.954803  0.0345429   0.0106539
 0.980443  0.0184453   0.00111201
 0.991519  0.00806756  0.000413186

In [72]:
returns[1:5,:]

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,Mexico,0.549451,2.95,6.12,0.64539,0.253165,0.140449
2,Argentina,Saudi Arabia,0.120048,8.21,24.52,0.892819,0.108578,0.039185
3,Australia,Denmark,6.38,3.88,0.429185,0.135501,0.204918,0.6997
4,Belgium,Canada,0.900901,2.45,3.51,0.526066,0.289855,0.221729
5,Belgium,Morocco,0.581395,3.21,4.92,0.632353,0.23753,0.168919


In [73]:
# check if each row of new_xgb_preds and returns are the same
for i in 1:size(new_xgb_preds,1)
    if new_xgb_preds[i,1] != returns[i,1] || new_xgb_preds[i,2] != returns[i,2]
        println("not the same")
    end
end


## ____________________________________________________________________

In [74]:
function FIFA2022_GS_Bettings(P, R, B, K)
    
    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = size(P, 1)              # 48 group stage matches
    n = size(P, 2)              # 3 outcomes

    @variable(model, x[i=1:m, j=1:n] >= 0) # x[i,j] is the amount of money bet on outcome j of match i
    @variable(model, z[i=1:m, j=1:n], Bin) # z[i,j] is whether you bet on outcome j of match i

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(P[i,1]*R[i,1]*x[i,1] + 
                               P[i,2]*R[i,2]*x[i,2] + 
                               P[i,3]*R[i,3]*x[i,3] - 
                               sum(x[i,j] for j=1:n) for i=1:m))

    # # constraints
    # 1) You can only bet on one outcome for each match
    @constraint(model, [i=1:m], sum(z[i,j] for j=1:n) <= 1)

    # 2) x[i,j] <= B/12*z[i,j] for all i and j
    # you can't bet more than half of your budget on a single match
    @constraint(model, [i=1:m, j=1:n], x[i,j] <= B/20*z[i,j])

    # 3) Budget constraints
    @constraint(model, sum(x[i,j] for j=1:n, i=1:m) <= B)

    # 4) You must spread your risk across at least K matches
    @constraint(model, sum(z[i,j] for j=1:n, i=1:m) >= K)

    # 5) if z = 1, then x > 0
    @constraint(model, [i=1:m, j=1:n], x[i,j] >= 1*z[i,j])

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings (generic function with 1 method)

In [75]:
exp_prof, x_opt, z_opt = FIFA2022_GS_Bettings(new_xgb_probs, returns_matrix, 5000, 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-16
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 338 rows, 288 columns and 1008 nonzeros
Model fingerprint: 0x3a294fae
Variable types: 144 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [7e-02, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 1951.0988708
Presolve removed 243 rows and 178 columns
Presolve time: 0.00s
Presolved: 95 rows, 110 columns, 344 nonzeros
Variable types: 0 continuous, 110 integer (79 binary)
Found heuristic solution: objective 2034.4685225

Root relaxation: objective 2.245316e+04, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe

In [76]:
# which matches to bet on
betting_matches = findall(x -> x >= 1, z_opt)

20-element Vector{CartesianIndex{2}}:
 CartesianIndex(3, 1)
 CartesianIndex(8, 1)
 CartesianIndex(9, 1)
 CartesianIndex(10, 1)
 CartesianIndex(11, 1)
 CartesianIndex(12, 1)
 CartesianIndex(24, 1)
 CartesianIndex(26, 1)
 CartesianIndex(31, 1)
 CartesianIndex(35, 1)
 CartesianIndex(37, 1)
 CartesianIndex(38, 1)
 CartesianIndex(6, 2)
 CartesianIndex(20, 2)
 CartesianIndex(19, 3)
 CartesianIndex(23, 3)
 CartesianIndex(33, 3)
 CartesianIndex(40, 3)
 CartesianIndex(42, 3)
 CartesianIndex(43, 3)

In [77]:
# make betting matches into list of first index of tuples
betting_matches_list = [x[1] for x in betting_matches]
betting_matches_outcomes = [x[2] for x in betting_matches]

betting_matches_list

20-element Vector{Int64}:
  3
  8
  9
 10
 11
 12
 24
 26
 31
 35
 37
 38
  6
 20
 19
 23
 33
 40
 42
 43

In [78]:
outcomes = new_xgb_preds[betting_matches_list,:]

Row,home_team,away_team,home_win,draw,home_loss
,String15,String15,Float64,Float64,Float64
1,Australia,Denmark,0.954803,0.0345429,0.0106539
2,Cameroon,Brazil,0.942382,0.0509663,0.00665153
3,Cameroon,Serbia,0.891956,0.0729,0.035144
4,Canada,Morocco,0.88168,0.0102742,0.108046
5,Costa Rica,Germany,0.978623,0.0184663,0.00291089
6,Croatia,Belgium,0.932715,0.0640844,0.00320028
7,Japan,Spain,0.989969,0.00867209,0.00135936
8,Korea Republic,Portugal,0.912574,0.0279381,0.0594874
9,Poland,Argentina,0.99983,0.000126039,4.36873e-5


In [79]:
outcomes[:,"bet_amount"] = [x_opt[i] for i in betting_matches];

In [80]:
real_outcomes = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\group_stage_mapped_outcomes.csv",DataFrame);

In [81]:
size(real_outcomes)[1]

48

In [82]:
#[real_outcomes[i,4] for i betting_matches_list]


In [83]:
# rf_pred_outcomes
outcomes[:,"actual_outcome"] = [real_outcomes[i,4] for i in betting_matches_list];

# # new_xgb_pred_outcomes (fill out)
# outcomes[:,"actual_outcome"]=[]

20-element Vector{Int64}:
 1
 1
 2
 3
 3
 2
 1
 1
 3
 3
 3
 3
 1
 3
 1
 3
 1
 1
 1
 3

In [84]:
outcomes[:,"bet_outcome"] = [x[2] for x in betting_matches];

In [85]:
# is actual_outcome = bet_outcome?
outcomes[:,"correct"] = outcomes[:,"actual_outcome"] .== outcomes[:,"bet_outcome"];

In [86]:
# bet result: if correct == false, then bet_result = -bet_amount, if correct == true, then bet_result = bet_amount*
bet_result = exp_prof - sum(outcomes[:,"bet_amount"].* (outcomes[:,"correct"].==false))

18953.161510600003

In [87]:
-sum(outcomes[:,"bet_amount"].*(outcomes[:,"correct"].==false)) # loss

-3500.0

In [88]:
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool
1,Australia,Denmark,0.954803,0.0345429,0.0106539,250.0,1,1,true
2,Cameroon,Brazil,0.942382,0.0509663,0.00665153,250.0,1,1,true
3,Cameroon,Serbia,0.891956,0.0729,0.035144,250.0,2,1,false
4,Canada,Morocco,0.88168,0.0102742,0.108046,250.0,3,1,false
5,Costa Rica,Germany,0.978623,0.0184663,0.00291089,250.0,3,1,false
6,Croatia,Belgium,0.932715,0.0640844,0.00320028,250.0,2,1,false
7,Japan,Spain,0.989969,0.00867209,0.00135936,250.0,1,1,true
8,Korea Republic,Portugal,0.912574,0.0279381,0.0594874,250.0,1,1,true
9,Poland,Argentina,0.99983,0.000126039,4.36873e-5,250.0,3,1,false


In [89]:
betting_matches

20-element Vector{CartesianIndex{2}}:
 CartesianIndex(3, 1)
 CartesianIndex(8, 1)
 CartesianIndex(9, 1)
 CartesianIndex(10, 1)
 CartesianIndex(11, 1)
 CartesianIndex(12, 1)
 CartesianIndex(24, 1)
 CartesianIndex(26, 1)
 CartesianIndex(31, 1)
 CartesianIndex(35, 1)
 CartesianIndex(37, 1)
 CartesianIndex(38, 1)
 CartesianIndex(6, 2)
 CartesianIndex(20, 2)
 CartesianIndex(19, 3)
 CartesianIndex(23, 3)
 CartesianIndex(33, 3)
 CartesianIndex(40, 3)
 CartesianIndex(42, 3)
 CartesianIndex(43, 3)

## money actually earned

In [90]:
outcomes

# add betting_matches_list as a column called :betting_indices so we can map to the correct row and columns in 'returns' dataframe
outcomes[:,"betting_matches_indices"] = betting_matches_list;
outcomes[:,"betting_matches_outcomes"] = betting_matches_outcomes;
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64
1,Australia,Denmark,0.954803,0.0345429,0.0106539,250.0,1,1,true,3,1
2,Cameroon,Brazil,0.942382,0.0509663,0.00665153,250.0,1,1,true,8,1
3,Cameroon,Serbia,0.891956,0.0729,0.035144,250.0,2,1,false,9,1
4,Canada,Morocco,0.88168,0.0102742,0.108046,250.0,3,1,false,10,1
5,Costa Rica,Germany,0.978623,0.0184663,0.00291089,250.0,3,1,false,11,1
6,Croatia,Belgium,0.932715,0.0640844,0.00320028,250.0,2,1,false,12,1
7,Japan,Spain,0.989969,0.00867209,0.00135936,250.0,1,1,true,24,1
8,Korea Republic,Portugal,0.912574,0.0279381,0.0594874,250.0,1,1,true,26,1
9,Poland,Argentina,0.99983,0.000126039,4.36873e-5,250.0,3,1,false,31,1


In [91]:
# # EXPECTED PROFIT
# create a new column called "if_bets_correct_profit" that calculates the profit if the bets are correct
# initialise new if bets correct profit list
if_bets_correct_profit = zeros(size(outcomes,1))

# for each row in outcomes, calculate the profit if the bets are correct
for i in 1:size(outcomes,1)
    if_bets_correct_profit[i] = outcomes[i,"bet_amount"] * returns[outcomes[i,"betting_matches_indices"],outcomes[i,"betting_matches_outcomes"]+2] - outcomes[i,"bet_amount"]
end

# add if_bets_correct_profit to outcomes dataframe
outcomes[:,"if_bets_correct_profit"] = if_bets_correct_profit;
outcomes


Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes,if_bets_correct_profit
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64,Float64
1,Australia,Denmark,0.954803,0.0345429,0.0106539,250.0,1,1,true,3,1,1345.0
2,Cameroon,Brazil,0.942382,0.0509663,0.00665153,250.0,1,1,true,8,1,1732.5
3,Cameroon,Serbia,0.891956,0.0729,0.035144,250.0,2,1,false,9,1,825.0
4,Canada,Morocco,0.88168,0.0102742,0.108046,250.0,3,1,false,10,1,695.0
5,Costa Rica,Germany,0.978623,0.0184663,0.00291089,250.0,3,1,false,11,1,5447.5
6,Croatia,Belgium,0.932715,0.0640844,0.00320028,250.0,2,1,false,12,1,255.0
7,Japan,Spain,0.989969,0.00867209,0.00135936,250.0,1,1,true,24,1,1260.0
8,Korea Republic,Portugal,0.912574,0.0279381,0.0594874,250.0,1,1,true,26,1,565.0
9,Poland,Argentina,0.99983,0.000126039,4.36873e-5,250.0,3,1,false,31,1,1547.5


In [92]:
# # ACTUAL PROFIT
# for each row i in outcomes:
# if correct == true, then actual_profit = if_bets_correct_profit, 
# if correct == false, then actual_profit = -bet_amount

# initialise new actual profit list
actual_profit = zeros(size(outcomes,1))

# for each row in outcomes, calculate the actual profit and store in actual_profit list
for i in 1:size(outcomes,1)
    if outcomes[i,"correct"] == true
        actual_profit[i] = outcomes[i,"if_bets_correct_profit"]
    else
        actual_profit[i] = -outcomes[i,"bet_amount"]
    end
end

# add actual_profit to outcomes dataframe
outcomes[:,"actual_profit"] = actual_profit;
outcomes

Row,home_team,away_team,home_win,draw,home_loss,bet_amount,actual_outcome,bet_outcome,correct,betting_matches_indices,betting_matches_outcomes,if_bets_correct_profit,actual_profit
,String15,String15,Float64,Float64,Float64,Float64,Int64,Int64,Bool,Int64,Int64,Float64,Float64
1,Australia,Denmark,0.954803,0.0345429,0.0106539,250.0,1,1,true,3,1,1345.0,1345.0
2,Cameroon,Brazil,0.942382,0.0509663,0.00665153,250.0,1,1,true,8,1,1732.5,1732.5
3,Cameroon,Serbia,0.891956,0.0729,0.035144,250.0,2,1,false,9,1,825.0,-250.0
4,Canada,Morocco,0.88168,0.0102742,0.108046,250.0,3,1,false,10,1,695.0,-250.0
5,Costa Rica,Germany,0.978623,0.0184663,0.00291089,250.0,3,1,false,11,1,5447.5,-250.0
6,Croatia,Belgium,0.932715,0.0640844,0.00320028,250.0,2,1,false,12,1,255.0,-250.0
7,Japan,Spain,0.989969,0.00867209,0.00135936,250.0,1,1,true,24,1,1260.0,1260.0
8,Korea Republic,Portugal,0.912574,0.0279381,0.0594874,250.0,1,1,true,26,1,565.0,565.0
9,Poland,Argentina,0.99983,0.000126039,4.36873e-5,250.0,3,1,false,31,1,1547.5,-250.0


In [93]:
# sum outcomes[:,"actual_earnings"]
sum(outcomes[:,"actual_profit"])

3150.0